In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os

# Specific plotting style to make the plots look even nicer
plt.style.use("bmh")

### Loading the data

In [ ]:
# Loading processed data from the numpy file
dat = np.load("finaldata.npy")

### Splitting the data into a train and test set

In [ ]:
# Splitting the data for training and testing; data is also reshaped
window = 36 # The number of hours used as input and output; in our case samples of 36 hours
shift = 1*24 # Shifting the period of time used for training and testing; this is due to a larger 'window' resulting in not having data at the start of the train data
index_start = 12 + shift # Index of when training set should start; this is 1 September 2015 12:00; this is also shifted by one day
train_end_date = "01_02_2021_12" # End date for training set
test_start_date = "10_02_2021_12" # Start date for test set
train_end_ind = dat[:,0].tolist().index(train_end_date) +shift # Dates translated to index in the data
test_start_ind = dat[:,0].tolist().index(test_start_date) +shift
n_test = 7 # Number of days to predict

input_length = 24 # Timesteps; we are moving with 24 hours when going to a new sample
xtrain_length = int((train_end_ind-index_start)/input_length) # Total number of samples

# Defining the train data
X_train_temp = dat[index_start-(window-24):train_end_ind,1:].astype('float64')
y_train_temp = dat[index_start+input_length:train_end_ind+input_length+(window-24),1].astype('float64')

# Since we use the previous 36 hours to predict 36 hours ahead, each sample will have 12 hours repeated from the
# previous sample (36-24); thus the data is reshaped in such a way that the 36 hours window is moved with steps of 24 hours
# through the data and then stacked
X_train = np.array([X_train_temp[0:window]])
y_train = y_train_temp[0:window]
for i in range(1,xtrain_length):
    X_train = np.vstack((X_train,np.array([X_train_temp[i*24:i*24+window]])))
    y_train = np.vstack((y_train,y_train_temp[i*24:i*24+window]))
y_train = np.expand_dims(y_train, axis=2)

# Defining the test data
X_test_temp = dat[test_start_ind-(window-24):test_start_ind + n_test*input_length,1:].astype('float64')
y_test_temp = dat[test_start_ind+input_length:test_start_ind + n_test*input_length + input_length+(window-24),1].astype('float64')

# Same as for the training data, where the 36 hours window is stacked for steps of 24 hours
X_test = np.array([X_test_temp[0:window]])
y_test = y_test_temp[0:window]
for i in range(1,n_test):
    X_test = np.vstack((X_test,np.array([X_test_temp[i*24:i*24+window]])))
    y_test = np.vstack((y_test,y_test_temp[i*24:i*24+window]))
y_test = np.expand_dims(y_test, axis=2)

print("Input train shape",X_train.shape)
print("Output train shape",y_train.shape)

print("Input test shape",X_test.shape)
print("Output test shape",y_test.shape)

### Configuring the layers

In [ ]:
# Function which returns the configuration of the network
def config_layers(X_train, NNH_1, NNH_2,NNH_3, y_train):
    num_input_units = np.shape(X_train)[1] # Number of inputs
    NNH_1 = NNH_1 # Number of hidden neurons for layer 1
    NNH_2 = NNH_2 # Number of hidden neurons for layer 2
    NNH_2 = NNH_3 # Number of hidden neurons for layer 3
    num_logits = np.shape(y_train)[1] # Number of logits
    return num_input_units, NNH_1, NNH_2,NNH_3, num_logits

### Defining the quantile losses

In [ ]:
# Function defining the quantile loss
def quantile_loss(q, y_p, y):
        e = y_p-y
        return tf.keras.backend.mean(tf.keras.backend.maximum(q*e, (q-1)*e))

# Defining the different quantiles and putting them together
loss_q_1 = lambda y,f: quantile_loss(0.01,y,f)
loss_q_5 = lambda y,f: quantile_loss(0.05,y,f)
loss_q_10 = lambda y,f: quantile_loss(0.10,y,f)
loss_q_25 = lambda y,f: quantile_loss(0.25,y,f)
loss_q_50 = lambda y,f: quantile_loss(0.50,y,f)
loss_q_75 = lambda y,f: quantile_loss(0.75,y,f)
loss_q_90 = lambda y,f: quantile_loss(0.90,y,f)
loss_q_95 = lambda y,f: quantile_loss(0.95,y,f)
loss_q_99 = lambda y,f: quantile_loss(0.99,y,f)

quantile_loss_array = [loss_q_1, loss_q_5, loss_q_10, loss_q_25, loss_q_50, loss_q_75, loss_q_90, loss_q_95, loss_q_99]

### Creating and training the model

In [ ]:
# Getting the configuration for the different layers of the network
num_input_units, NNH_1, NNH_2,NNH_3, num_logits = config_layers(X_train,19,20,19,y_train)

In [ ]:
# Defining the hyperparameters of the network
learning_rate = 0.001
epochs = 300
validation_split = 0.25
batch_size = 125

# Defining the optimizer
optimizer = keras.optimizers.Adam(lr=learning_rate)

# Defining the layers; LSTM layers are all bidirectional
input_layer = keras.layers.Input(shape=(num_input_units, 7))
lstm1_layer = keras.layers.Bidirectional(keras.layers.LSTM(NNH_1), input_shape=(num_input_units, 7))(input_layer)
repeat_layer = keras.layers.RepeatVector(num_logits)(lstm1_layer) # Repeats its input 36 times to get back to original 3D shape
lstm2_layer = keras.layers.Bidirectional(keras.layers.LSTM(NNH_2, return_sequences=True, activation='softplus'))(repeat_layer)
lstm3_layer = keras.layers.Bidirectional(keras.layers.LSTM(NNH_3, return_sequences=True, activation='softplus'))(lstm2_layer)
# Dropout layer for regularization
dropout_layer = keras.layers.Dropout(0.20)(lstm3_layer)
# Each quantile uses the previously defined layers to make sure that they use the same network to get to the loss
quantile_output_q_1 = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='softplus'))(dropout_layer)
quantile_output_q_5 = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='softplus'))(dropout_layer)
quantile_output_q_10 = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='softplus'))(dropout_layer)
quantile_output_q_25 = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='softplus'))(dropout_layer)
quantile_output_q_50 = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='softplus'))(dropout_layer)
quantile_output_q_75 = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='softplus'))(dropout_layer)
quantile_output_q_90 = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='softplus'))(dropout_layer)
quantile_output_q_95 = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='softplus'))(dropout_layer)
quantile_output_q_99 = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='softplus'))(dropout_layer)

# Defining the model
model = keras.models.Model(inputs=input_layer, outputs=[quantile_output_q_1,quantile_output_q_5,
                                                        quantile_output_q_10,quantile_output_q_25,
                                                        quantile_output_q_50,quantile_output_q_75,
                                                        quantile_output_q_90,quantile_output_q_95,quantile_output_q_99])

# Defining the compiler with the loss function and the optimizer
model.compile(loss=quantile_loss_array , optimizer=optimizer)

# Callback for keeping the best iteration of the model during training
checkpoint_path = "learned_weights/weights.best.hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     monitor='val_loss',
                                                     mode='auto',
                                                     save_best_only=True,
                                                     save_weights_only=False,
                                                     save_freq='epoch',
                                                     verbose=1)

# Fitting the model
history = model.fit(X_train, [y_train]*9, validation_split=validation_split,\
                    epochs=epochs, batch_size=batch_size, callbacks = [cp_callback])

### Plotting the model loss history

In [ ]:
# Summarizes the losses for the epochs
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

### Saving the model

In [ ]:
# Uncomment the line of code below to save the trained model

# model.save('my_awesome_model')

### Loading the model

In [ ]:
# Uncomment the line of code below to load a previously trained model
# !!!Make sure to run the first 3 cells of this file before loading the model!!!

# model = keras.models.load_model('final_model_epoch300_lr0_0005_vs0_25_bs125', compile = False)

### Predicting on the test set

In [ ]:
# Prediciting the output on our test set
y_pred = model.predict(X_test)
y_pred = np.asarray(y_pred) # Converting to a Numpy array
print(y_pred.shape)

### Calculating the RMSE error and the quantile mean loss

In [ ]:
# Calculating the RMSE error between the predicted quantile 50 and the actual data
rmse_error = np.sqrt(np.mean((y_pred[4,:,12:36].flatten()-y_test[:,12:36].flatten())**2))

#Calculating the mean quantile loss:
quantiles = [0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]
qloss = 0 
for i in range(len(y_pred)):
    e = y_pred[i,:,12:36].flatten()-y_test[:,12:36].flatten()
    currentqloss = np.mean(np.maximum(quantiles[i]*(e), (quantiles[i]-1)*e))
    qloss = qloss + currentqloss
    print("Average quantile loss for quantile",quantiles[i], ":",currentqloss)

quantile_mean_loss = qloss/len(quantiles)
print('RMSE error =', rmse_error)
print("Quantile mean loss =",quantile_mean_loss)

### Plotting the results

In [ ]:
# Plotting the four regions of the quantiles, the actual value and the predicted value (quantile 50)
plt.figure(figsize=(20, 6))
plt.fill_between(range(24*7),y_pred[0,:,12:36].flatten(),y_pred[-1,:,12:36].flatten(), color = "#999999", label = "quantile 1-99")
plt.fill_between(range(24*7),y_pred[1,:,12:36].flatten(),y_pred[-2,:,12:36].flatten(), color = "#808080", label = "quantile 5-95")
plt.fill_between(range(24*7),y_pred[2,:,12:36].flatten(),y_pred[-3,:,12:36].flatten(), color = "#666666", label = "quantile 10-90")
plt.fill_between(range(24*7),y_pred[3,:,12:36].flatten(),y_pred[-4,:,12:36].flatten(), color = "#4d4d4d", label = "quantile 25-75")
plt.plot(y_test[:,12:36].flatten(),color="red", label = "actual value")
plt.plot(y_pred[4,:,12:36].flatten(),color="blue", label = "predicted value")

# Plotting vertical lines to seperate the days
plt.axvline(x=24-1, color="black", linestyle="--")
plt.axvline(x=24*2-1, color="black", linestyle="--")
plt.axvline(x=24*3-1, color="black", linestyle="--")
plt.axvline(x=24*4-1, color="black", linestyle="--")
plt.axvline(x=24*5-1, color="black", linestyle="--")
plt.axvline(x=24*6-1, color="black", linestyle="--")
plt.axvline(x=24*7-1, color="black", linestyle="--")

plt.xticks([11,35,59,83,107,131,155],['Day 1', 'Day 2','Day 3', 'Day 4', 'Day 5', 'Day 6', 'Day 7'])
plt.xlim(0,24*7-1)
plt.ylabel("Day-ahead prices [Euros]")
plt.legend()
plt.show()